## C02.01.F4. Train Model. Smoothed dataset. All Features

# Script for training the sellout extrapolation algorithm.

The training dataset is divided in two to have a new training dataset and the 
validation dataset. On the new training dataset the gridsearch is adjusted to 
obtain the best parameters that reduce the error when checking the results on 
the validation dataset. The cross validation error is also checked for the 
new training dataset.

In [1]:
# Import libraries 
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.model_selection import RandomizedSearchCV, GroupKFold, cross_val_score
import lightgbm as ltb
import matplotlib.pyplot as plt 
import joblib
import time 
import warnings
warnings.filterwarnings("ignore")
import sys
t1 = time.time()

In [2]:
def eval_model(model, X, y, thresholds=[5], verbose=1):
    y_pred = model.predict(X)
    y = np.power(y,3)
    y_pred = np.power(y_pred,3)
    porc_error = abs(y_pred - y)*100/y
    absolute_mean_error = mean_absolute_error(y, y_pred)
    porcentual_mean_error = np.mean(porc_error[porc_error != np.inf])
    if verbose:
        print("\nTEST: Absolute Error:", absolute_mean_error)
        print("Porcentual Error:", porcentual_mean_error)
        print("STD Error:", np.std(abs(y - y_pred)))
        print("R2 Score:", r2_score(y, y_pred)) #Coef determ
    results = pd.DataFrame(np.array(y), columns = ['REAL'])
    results['PRED'] = y_pred
    results['PERCENTAGE_ERROR'] = np.abs(results['PRED'] - results['REAL'])*100/results['REAL'] 
    results['ABSOLUTE_ERROR'] = np.abs(results['PRED'] - results['REAL'])*100
    results['R2_SCORE'] = r2_score(results['REAL'], results['PRED'])
    
    for threshold in thresholds:
        hits = 0
        for element in results['ABSOLUTE_ERROR']:
            if element <= threshold:
                hits+=1
        
        porcentual_hits = hits*100/len(results)
        if verbose:
            print(str(porcentual_hits)+str("% registers are with less than"), str(threshold)+str("% of absolute error."))
            results['% REG ERROR < '+str(threshold)] = porcentual_hits
        
    return results

In [3]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import RANSACRegressor

from sklearn.tree import DecisionTreeRegressor

from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.gaussian_process.kernels import RBF
from sklearn.gaussian_process import GaussianProcessClassifier


from sklearn.linear_model import LinearRegression

from sklearn.gaussian_process.kernels import RationalQuadratic





from sklearn import model_selection
from sklearn.utils import class_weight
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import numpy as np
import pandas as pd

In [4]:
# Define INPUT and OUTPUT files
INPUT = '../../F3.Data Preparation/02_Data/sampled_train.csv'

OUTPUT_MODEL = '../02_Data/AllFeat_model.pkl'
OUTPUT_SC2 = '../02_Data/AllFeat_sc_X2.bin'
OUTPUT_TRAIN_RES = '../02_Data/AllFeat_train_results.csv'
OUTPUT_VAL_RES = '../02_Data/AllFeat_val_results.csv'

In [5]:
# Read data resetting the indexes
data = pd.read_csv(INPUT, sep='|').reset_index(drop=True)

In [6]:
# Indexing the independent and dependent variables in X and y respectively
# Create data copy by removing unnecessary columns in X and the sellout field in y
to_delete = ['R', 'CAL_DATE', 'CAL_DATE_end','SO_ITG_WSE', 'SO_MRKT_WSE','QUOTA_SELLOUT']
X = data.drop(to_delete, axis=1)
#Target QUOTA_SELLOUT
y = data['QUOTA_SELLOUT']

#### Given goal is to predict sales of customers for whom we have no Target data, the selection of train and test is done by choosing random customers, instead of random transactions.

In [7]:
#### Given goal is to predict sales of customers for whom we have no Target data, the selection of train and test is done by choosing random customers, instead of random transactions.# Getting unique customers and shuffle them
customers = X['CUSTOMER_ID'].drop_duplicates().reset_index(drop=True)
index = np.random.permutation(len(customers))
customers = customers.loc[index].reset_index(drop=True)

In [8]:
# Separate train and val customers by ratio
ratio = 0.9
train = customers[:int(np.round(len(customers)*ratio))]
val = customers[int(np.round(len(customers)*ratio)):]

In [9]:
val

4925     8950701
4926     2020121
4927     6000210
4928    46040622
4929    18070078
          ...   
5467    46001897
5468    12000071
5469    18000430
5470    43021054
5471    13020094
Name: CUSTOMER_ID, Length: 547, dtype: int64

In [10]:
train

0       32041337
1        6150296
2        3010634
3       23020126
4        2000370
          ...   
4920    45040037
4921    10090363
4922    25000019
4923    11000320
4924    14000897
Name: CUSTOMER_ID, Length: 4925, dtype: int64

In [11]:
# Obtain the indexes of the dataset where the customers are located
index_train = X['CUSTOMER_ID'].isin(train)
index_val = X['CUSTOMER_ID'].isin(val)

X_train = X[index_train].reset_index(drop=True)
X_val = X[index_val].reset_index(drop=True)

y_train = y[index_train].reset_index(drop=True)
y_val = y[index_val].reset_index(drop=True)

In [12]:
# Remove CUSTOMER_ID from data
X_train = X_train.drop(['CUSTOMER_ID','BRANDFAMILY_ID'], axis=1)
X_val = X_val.drop(['CUSTOMER_ID','BRANDFAMILY_ID'], axis=1)
X = X.drop(['CUSTOMER_ID','BRANDFAMILY_ID'], axis=1)

In [13]:
# Apply a data scaling with StandardScaler on X_train and transform on X_val
sc_X = StandardScaler()
X_train = sc_X.fit_transform(X_train)
X_val = sc_X.transform(X_val)

In [14]:
# Save StandardScaler configuration to file for future use
joblib.dump(sc_X, OUTPUT_SC2, compress=True)

['../02_Data/AllFeat_sc_X2.bin']

In [15]:
# Transform train and val dependent variable to a gaussian distribution
y_train = y_train**(float(1)/3)
y_val = y_val**(float(1)/3)

In [20]:
def run_exps(X_train:pd.DataFrame,y_train:pd.DataFrame,X_test:pd.DataFrame,y_test:pd.DataFrame) -> pd.DataFrame:
    '''
    Lightweight script to test many models and find winners
    :param X_train: training split
    :param y_train: training target vector
    :param X_test: test split
    :param y_test: test target vector
    :return: DataFrame of predictions
    '''
    kRBF =1.0*RBF( length_scale=1.0 , length_scale_bounds=(1e-1, 10.0))    
    kQuadratic = 1.0*RationalQuadratic ( length_scale=1.0 , alpha=0.1)
    dfs = []
    models = [
            ####  ('LogReg', LogisticRegression())##, 
            ####  ('Poly', PolynomialFeatures())
            ####  ('RanSac', RANSACRegressor(LinearRegression(), max_trials=4, min_samples=2,loss='absolute_loss',residual_threshold=10)) 
            ## ('linear', LinearRegression() ),
           ## ('SVR', SVR(C=1.0, epsilon=0.2, kernel='rbf')),          
            ##  ('Forest', RandomForestRegressor(n_estimators=20,max_depth=10,criterion='mse', )), 
            ##  ('Tree', DecisionTreeRegressor(max_depth=1)),
              ('GPCR',GaussianProcessClassifier(kernel=kRBF, random_state=0)),
             ##  ('GPCQ',GaussianProcessClassifier(kernel=kQuadratic, random_state=0))          
            ]
    results = []
    names = []
    scoring = ['accuracy', 'precision_weighted', 'recall_weighted', 'f1_weighted', 'roc_auc']

    #target_names = ['malignant', 'benign']

    for name, model in models:
            kfold = model_selection.KFold(n_splits=5, shuffle=True, random_state=90210)
            cv_results = model_selection.cross_validate(model, X_train, y_train, cv=kfold, scoring=scoring)
            clf = model.fit(X_train, y_train)
            y_pred = clf.predict(X_test)
            print(name)
            print(regression_report(y_test, y_pred, target_names=target_names))
    results.append(cv_results)
    names.append(name)
    this_df = pd.DataFrame(cv_results)
    this_df['model'] = name
    dfs.append(this_df)
    final = pd.concat(dfs, ignore_index=True)
    return final

In [21]:
final = run_exps(X_train , y_train, X_val, y_val)

ValueError: Unknown label type: (array([0.23431064, 0.21622342, 0.3139285 , ..., 0.57490584, 0.55549208,
       0.58273284]),)

In [ ]:
bootstraps = []
for model in list(set(final.model.values)):
    model_df = final.loc[final.model == model]
    bootstrap = model_df.sample(n=30, replace=True)
    bootstraps.append(bootstrap)
        
bootstrap_df = pd.concat(bootstraps, ignore_index=True)
results_long = pd.melt(bootstrap_df,id_vars=['model'],var_name='metrics', value_name='values')
time_metrics = ['fit_time','score_time'] # fit time metrics
## PERFORMANCE METRICS
results_long_nofit = results_long.loc[~results_long['metrics'].isin(time_metrics)] # get df without fit data
results_long_nofit = results_long_nofit.sort_values(by='values')
## TIME METRICS
results_long_fit = results_long.loc[results_long['metrics'].isin(time_metrics)] # df with fit data
results_long_fit = results_long_fit.sort_values(by='values')

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
plt.figure(figsize=(20, 12))
sns.set(font_scale=2.5)
g = sns.boxplot(x="model", y="values", hue="metrics", data=results_long_nofit, palette="Set3")
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.title('Comparison of Model by Classification Metric')
plt.savefig('./benchmark_models_performance.png',dpi=300)

In [ ]:
plt.figure(figsize=(20, 12))
sns.set(font_scale=2.5)
g = sns.boxplot(x="model", y="values", hue="metrics", data=results_long_fit, palette="Set3")
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.title('Comparison of Model by Fit and Score Time')
plt.savefig('./benchmark_models_time.png',dpi=300)

In [ ]:
metrics = list(set(results_long_nofit.metrics.values))
bootstrap_df.groupby(['model'])[metrics].agg([np.std, np.mean])

In [ ]:
time_metrics = list(set(results_long_fit.metrics.values))
bootstrap_df.groupby(['model'])[time_metrics].agg([np.std, np.mean])

In [ ]:
# Save model configuration to file for future use
joblib.dump(model, OUTPUT_MODEL)

In [ ]:
# Get results from evaluate the model with train and val data
res_train = eval_model(model, X_train, y_train, [1, 2, 5, 10], verbose=1)
res_val = eval_model(model, X_val, y_val, [1, 2, 5, 10], verbose=1)

##### Good result with an R2 of 90%.

# Format results dataframes for write to file
res_train = data[index_train][['CUSTOMER_ID','BRANDFAMILY_ID','CAL_DATE','CAL_DATE_end']].reset_index(drop=True).merge(res_train, left_index=True, right_index=True)
res_val = data[index_val][['CUSTOMER_ID','BRANDFAMILY_ID','CAL_DATE','CAL_DATE_end']].reset_index(drop=True).merge(res_val, left_index=True, right_index=True)

res_train['CV_R2_SCORE'] = scores.mean()

# The data is written to a file
res_train.to_csv(OUTPUT_TRAIN_RES, sep='|', index=False)
res_val.to_csv(OUTPUT_VAL_RES, sep='|', index=False)

#### Smoothing makes the impact of the Quota Sell In much greater and differentiated from the rest.

In [ ]:
t2 = time.time()
print ("Time to execute script:",str(round((t2-t1)/3600,2)), "h")